In [36]:
# Load all the important libraries to create the RandomForestClassifier Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [37]:
#import the data set

Data = pd.read_csv(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Dataset\CKD.csv")

In [38]:
Data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [39]:
#Lets check total number of rows and columns 
Data.shape

(399, 25)

In [40]:
#Check the dataset info 
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             399 non-null    float64
 1   bp              399 non-null    float64
 2   sg              399 non-null    object 
 3   al              399 non-null    float64
 4   su              399 non-null    float64
 5   rbc             399 non-null    object 
 6   pc              399 non-null    object 
 7   pcc             399 non-null    object 
 8   ba              399 non-null    object 
 9   bgr             399 non-null    float64
 10  bu              399 non-null    float64
 11  sc              399 non-null    float64
 12  sod             399 non-null    float64
 13  pot             399 non-null    float64
 14  hrmo            399 non-null    float64
 15  pcv             399 non-null    float64
 16  wc              399 non-null    float64
 17  rc              399 non-null    flo

In [41]:
# Lets check the description of data set
Data.describe()

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc
count,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000
mean,51.492308,76.459948,0.899749,0.395990,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597
std,16.995379,13.492053,1.314769,1.041155,74.864224,49.336046,5.623758,9.215829,2.823323,2.715753,8.157274,2526.204544,0.841006
min,2.000000,50.000000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000,9.000000,2200.000000,2.100000
25%,42.000000,70.000000,0.000000,0.000000,101.000000,27.000000,0.900000,135.000000,4.000000,10.850000,34.000000,6950.000000,4.500000
50%,54.000000,76.459948,0.000000,0.000000,127.000000,44.000000,1.400000,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597
75%,64.000000,80.000000,2.000000,0.000000,150.000000,62.500000,3.077356,141.000000,4.800000,14.600000,44.000000,9400.000000,5.100000
max,90.000000,180.000000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000,54.000000,26400.000000,8.000000


In [42]:
# Lets check if the data set has NA values
Data[Data.isna().any(axis=1)]

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification


In [43]:
# Lets check if the data set has Null values
Data[Data.isnull().any(axis=1)]

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification


In [44]:
# Lets check how many classes are there in the target variable "Classification"
Data.classification.value_counts()

yes    249
no     150
Name: classification, dtype: int64

# From above output we got to know that there are 250 records in "Yes" class and 150 records in "No" class. so its called as imbalanced data set

In [45]:
# Lets upsample the "No" records to match with "Yes" records
Data_No = Data[Data["classification"]=="no"]

In [46]:
import random

samples_index = random.sample(range(150),99)

In [47]:
Data_No_final = Data_No.iloc[samples_index]

In [48]:
Data_No_final.reset_index(inplace=True,drop=True)

In [49]:
Data_No_final

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,30.0,60.0,a,0.0,0.0,normal,normal,notpresent,notpresent,138.0,15.0,1.1,135.0,4.4,12.518156,38.868902,8408.191126,4.705597,no,no,no,yes,poor,no,no
1,47.0,60.0,a,0.0,0.0,normal,normal,notpresent,notpresent,137.0,17.0,0.5,150.0,3.5,13.600000,44.000000,7900.000000,4.500000,no,no,no,yes,poor,no,no
2,36.0,80.0,b,0.0,0.0,normal,normal,notpresent,notpresent,85.0,16.0,1.1,142.0,4.1,15.600000,44.000000,5800.000000,6.300000,no,no,no,yes,poor,no,no
3,47.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,93.0,33.0,0.9,144.0,4.5,13.300000,52.000000,8100.000000,5.200000,no,no,no,yes,poor,no,no
4,29.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,70.0,16.0,0.7,138.0,3.5,13.700000,54.000000,5400.000000,5.800000,no,no,no,yes,poor,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,57.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,133.0,48.0,1.2,147.0,4.3,14.800000,46.000000,6600.000000,5.500000,no,no,no,yes,poor,no,no
95,45.0,80.0,b,0.0,0.0,normal,normal,notpresent,notpresent,82.0,49.0,0.6,147.0,4.4,15.900000,46.000000,9100.000000,4.700000,no,no,no,yes,poor,no,no
96,55.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,133.0,17.0,1.2,135.0,4.8,13.200000,41.000000,6800.000000,5.300000,no,no,no,yes,poor,no,no
97,19.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,107.0,23.0,0.7,141.0,4.2,14.400000,44.000000,8408.191126,4.705597,no,no,no,yes,poor,no,no


In [50]:
Data = Data.append(Data_No_final,ignore_index=True)

C:\Users\Vinoth\AppData\Local\Temp\ipykernel_15580\3974467208.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Data = Data.append(Data_No_final,ignore_index=True)


In [51]:
Data.classification.value_counts()

yes    249
no     249
Name: classification, dtype: int64

# From above output we have upsampled the "No" class counts to match with "Yes" counts

In [52]:
# Lets convert the nominal categoty columns to numerical columns using one-hot-encoding via pandas get_dummies method

Data = pd.get_dummies(Data,drop_first=True)

In [53]:
Data.head()

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc,sg_b,sg_c,sg_d,sg_e,rbc_normal,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.0,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1
1,3.0,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,34.000000,12300.000000,4.705597,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1
2,4.0,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,34.000000,8408.191126,4.705597,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1
3,5.0,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,38.868902,8408.191126,4.705597,0,0,1,0,1,1,0,0,0,0,0,1,0,1,1
4,5.0,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,36.000000,12400.000000,4.705597,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1


In [54]:
Data.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [55]:
# Lets seperate the dependent and independent variables
dependent = Data[['classification_yes']]
independent = Data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [56]:
import sklearn
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [58]:
# Lets spli the data to trai  and test set

X_Train,X_Test,Y_Train,Y_Test = train_test_split(independent,dependent,random_state=0,test_size=30)

In [28]:
# Initializing the model params for XGBoost Classifier to pass it as input for gridsearch CV
Model_Params = {"max_depth":[i for i in range(1,10,2)],'n_estimators':[50,100,300,500],"learning_rate":[0.1,0.001,0.001,1.0,10],
                "max_leaves":[i for i in range(8,32,2)],"booster":["gbtree","gblinear"],'subsample':[0.5],'gamma':[0.1,0.001,1,10]}

In [29]:
# Lets create a model now using the Gridsearch CV

XGBoost_Model_Creation = GridSearchCV(XGBClassifier(),Model_Params,n_jobs=-1,scoring={"roc_auc_score":'roc_auc','f1_weighted_score':'f1_weighted'},
                                            refit='roc_auc_score',verbose=2)

In [30]:
XGBoost_Model_Creation.fit(X_Train,Y_Train)

Fitting 5 folds for each of 9600 candidates, totalling 48000 fits


C:\Users\Vinoth\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [11:39:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves", "subsample" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'booster': ['gbtree', 'gblinear'],
                         'gamma': [0.1, 0.001, 1, 10],
                         'learning_rate': [0.1, 0.001, 0.001, 1.0, 10],
                         'max_depth': [1, 3, 5, 7, 9],
                         'max_leaves': [8, 10, 12, 14, 16, 18, 20, 22, 24, 26,
                                        28, 30],
                         'n_estimators': [50, 100, 300, 500],
                         'subsample': [0.5]},
             refit='roc_auc_score',
             scoring={'f1_weighted_score': 'f1_weighted',
                      'roc_auc_score': 'roc_auc'},
             verbose=2)

In [31]:
# The best parameter that has givem high performance scores
XGBoost_Model_Creation.best_params_

{'booster': 'gblinear',
 'gamma': 0.1,
 'learning_rate': 0.001,
 'max_depth': 1,
 'max_leaves': 8,
 'n_estimators': 300,
 'subsample': 0.5}

In [32]:
# The best score for the above parameters
XGBoost_Model_Creation.best_score_

0.9990828035506919

# Since there are lot of failures in the gridsearch CV results so the entries will be NA. so we don't need those entries to be saved as it may confuse us in future so we are removing those NA entries and have only the valid entries of grid search CV results saved and used for future reference

In [70]:
# Creating as data frame to save the GridSearch CV results
CV_Output = pd.DataFrame.from_dict(XGBoost_Model_Creation.cv_results_)

In [71]:
# Check the entries count
CV_Output.shape

(9600, 28)

In [72]:
# Final Data frame that has valid entries in it
CV_Output

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_gamma,param_learning_rate,param_max_depth,param_max_leaves,param_n_estimators,param_subsample,params,split0_test_roc_auc_score,split1_test_roc_auc_score,split2_test_roc_auc_score,split3_test_roc_auc_score,split4_test_roc_auc_score,mean_test_roc_auc_score,std_test_roc_auc_score,rank_test_roc_auc_score,split0_test_f1_weighted_score,split1_test_f1_weighted_score,split2_test_f1_weighted_score,split3_test_f1_weighted_score,split4_test_f1_weighted_score,mean_test_f1_weighted_score,std_test_f1_weighted_score,rank_test_f1_weighted_score
0,1.481716,0.071280,0.011202,0.003919,gbtree,0.1,0.1,1,8,50,0.5,"{'booster': 'gbtree', 'gamma': 0.1, 'learning_...",0.998642,1.0,0.995247,0.989362,1.0,0.996650,0.004037,4189,0.957370,1.000000,0.946754,0.903002,0.978480,0.957121,0.032653,5677
1,1.417596,0.042534,0.015608,0.000792,gbtree,0.1,0.1,1,8,100,0.5,"{'booster': 'gbtree', 'gamma': 0.1, 'learning_...",1.000000,1.0,0.998642,0.993525,1.0,0.998433,0.002510,2413,0.978714,1.000000,0.957428,0.935394,0.978480,0.970003,0.021925,4141
2,1.438187,0.026323,0.012821,0.003928,gbtree,0.1,0.1,1,8,300,0.5,"{'booster': 'gbtree', 'gamma': 0.1, 'learning_...",1.000000,1.0,0.999095,0.988899,1.0,0.997599,0.004364,3025,1.000000,1.000000,0.968081,0.956979,0.989245,0.982861,0.017416,2329
3,1.452968,0.041644,0.012806,0.003913,gbtree,0.1,0.1,1,8,500,0.5,"{'booster': 'gbtree', 'gamma': 0.1, 'learning_...",0.999547,1.0,0.998642,0.990287,1.0,0.997695,0.003737,2905,0.989360,1.000000,0.968081,0.967720,1.000000,0.985032,0.014518,1309
4,0.035215,0.008171,0.016005,0.005057,gbtree,0.1,0.1,1,10,50,0.5,"{'booster': 'gbtree', 'gamma': 0.1, 'learning_...",0.998642,1.0,0.995247,0.989362,1.0,0.996650,0.004037,4189,0.957370,1.000000,0.946754,0.903002,0.978480,0.957121,0.032653,5677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,0.286008,0.008219,0.013410,0.003309,gblinear,10,10,9,28,500,0.5,"{'booster': 'gblinear', 'gamma': 10, 'learning...",0.500000,0.5,0.500000,0.500000,0.5,0.500000,0.000000,8641,0.333333,0.333333,0.333333,0.339324,0.339324,0.335730,0.002935,8641
9596,0.041856,0.002975,0.017716,0.001981,gblinear,10,10,9,30,50,0.5,"{'booster': 'gblinear', 'gamma': 10, 'learning...",0.500000,0.5,0.500000,0.500000,0.5,0.500000,0.000000,8641,0.333333,0.333333,0.333333,0.339324,0.339324,0.335730,0.002935,8641
9597,0.069168,0.006520,0.017615,0.003192,gblinear,10,10,9,30,100,0.5,"{'booster': 'gblinear', 'gamma': 10, 'learning...",0.500000,0.5,0.500000,0.500000,0.5,0.500000,0.000000,8641,0.333333,0.333333,0.333333,0.339324,0.339324,0.335730,0.002935,8641
9598,0.174514,0.003198,0.014399,0.003207,gblinear,10,10,9,30,300,0.5,"{'booster': 'gblinear', 'gamma': 10, 'learning...",0.500000,0.5,0.500000,0.500000,0.5,0.500000,0.000000,8641,0.333333,0.333333,0.333333,0.339324,0.339324,0.335730,0.002935,8641


In [73]:
# Lets save the grid search CV results to the csv file
CV_Output.to_csv(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Grid Serach CV Results\XGBC_GridSerachCV_Results.csv")

In [74]:
# Lets test the model with best parameter that we have got agains the test data and check the performance

Y_Predicted = XGBoost_Model_Creation.predict(X_Test)

In [75]:
Y_Predicted

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1])

In [76]:
# Lets check the confusion matrix for the above predicted against the actual results
confusion_matrix(Y_Predicted,Y_Test)

array([[16,  0],
       [ 0, 14]], dtype=int64)

In [77]:
#lets check the classification test report for the predicted against the actual results
print(classification_report(Y_Predicted,Y_Test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        14

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [78]:
XGBoost_Model_Creation.predict_proba(X_Test)[:,1]

array([0.35363346, 0.25892282, 0.34010994, 0.3431297 , 0.6217493 ,
       0.2552984 , 0.7756221 , 0.26024988, 0.7883846 , 0.35405582,
       0.96748424, 0.667879  , 0.35663834, 0.94391984, 0.26688376,
       0.7566448 , 0.3545219 , 0.6738838 , 0.24896693, 0.70862126,
       0.35369214, 0.86516494, 0.6987028 , 0.5084394 , 0.35663834,
       0.3431835 , 0.29141375, 0.35228145, 0.8522194 , 0.8791139 ],
      dtype=float32)

In [79]:
# lets check the roc_auc_score results for the predicted against the actual results
roc_auc_score(Y_Test,XGBoost_Model_Creation.predict_proba(X_Test)[:,1])

1.0

In [80]:
# Wow we can see our auc score is 100 percent and weighted f1 score is also 93 percent lets save out model
import pickle
pickle.dump(XGBoost_Model_Creation,open(r'C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Final Model\XGBC_Final_Model.sav','wb'))

In [81]:
# Lets load and test our model
def Final_model_prod(model,columns,stdscaler='None'):
    query_values=[]
    for col_idx in range(0,len(columns)-1):
        query_values.append(float(input("Please enter valid {}: \n Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' ".format(columns[col_idx]))))
    pred_class=model.predict(stdscaler.transform([query_values]))
    
    if pred_class==0:
        print ("This patient doesn't have Chronic Kidney Disorder")
    else:
        print ("This patient have Chronic Kidney Disorder Please proceed proper medication")

In [82]:
# Load the saved model
Final_model= pickle.load(open(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Final Model\XGBC_Final_Model.sav",'rb'))

# here the scaler is the varaible that we have used for standardising the independent variables so we should use the same variable here  


In [ ]:
# Test the final model
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=UserWarning)
Final_model_prod(Final_model,Data.columns.to_list(),scaler)

In [ ]:
# Test the final model
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=UserWarning)
Final_model_prod(Final_model,Data.columns.to_list(),scaler)